# Hello Feature Class example: using the feature classes to calculate features

This example shows how to use the Radiomics package to directly instantiate the feature classes for feature extraction. 
Note that this is not the intended standard use. For an example on the standard use with feature extractor, see the `helloRadiomics` example.

In [1]:
from __future__ import print_function
import os
import collections
import SimpleITK as sitk
import numpy
import six
import radiomics
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm

## Getting the test case

Test cases can be downloaded to temporary files. This is handled by the `radiomics.getTestCase()` function, which checks if the requested test case is available and if not, downloads it. It returns a tuple with the location of the image and mask of the requested test case, or (None, None) if it fails.

Alternatively, if the data is available somewhere locally, this directory can be passed as a second argument to `radiomics.getTestCase()`. If that directory does not exist or does not contain the testcase, functionality reverts to default and tries to download the test data.

If getting the test case fails, PyRadiomics will log an error explaining the cause.

In [2]:
imageName, maskName = radiomics.getTestCase('brain1')

if imageName is None or maskName is None:  # Something went wrong, in this case PyRadiomics will also log an error
    raise Exception('Error getting testcase!')  # Raise exception to prevent cells below from running in case of "run all"

In [3]:
image = sitk.ReadImage(imageName)
mask = sitk.ReadImage(maskName)

## Preprocess the image

#### Extraction Settings

In [4]:
settings = {}
settings['binWidth'] = 25
settings['resampledPixelSpacing'] = None
# settings['resampledPixelSpacing'] = [3, 3, 3]  # This is an example for defining resampling (voxels with size 3x3x3mm)
settings['interpolator'] = 'sitkBSpline'
settings['verbose'] = True

#### If enabled, resample the image

In [5]:
# Resample if necessary
if settings['interpolator'] != None and settings['resampledPixelSpacing'] != None:
  image, mask = imageoperations.resampleImage(image, mask, settings['resampledPixelSpacing'], settings['interpolator'])

## Calculate features using original image

In [6]:
# Crop the image
# bb is the bounding box, upon which the image and mask are cropped
bb, correctedMask = imageoperations.checkMask(image, mask, label=1)
if correctedMask is not None:
    mask = correctedMask
croppedImage, croppedMask = imageoperations.cropToTumorMask(image, mask, bb)

### Calculate Firstorder features

In [7]:
firstOrderFeatures = firstorder.RadiomicsFirstOrder(croppedImage, croppedMask, **settings)

# Set the features to be calculated
firstOrderFeatures.enableFeatureByName('Mean', True)
# firstOrderFeatures.enableAllFeatures()

In [8]:
# Print out the docstrings of the enabled features
print('Will calculate the following first order features: ')
for f in firstOrderFeatures.enabledFeatures.keys():
  print(f)
  print(getattr(firstOrderFeatures, 'get%sFeatureValue' % f).__doc__)

Will calculate the following first order features: 
Mean

    **8. Mean**

    .. math::
      \textit{mean} = \frac{1}{N_p}\displaystyle\sum^{N_p}_{i=1}{\textbf{X}(i)}

    The average gray level intensity within the ROI.
    


In [9]:
# Calculate the features and print(out result)
print('Calculating first order features...',)
result = firstOrderFeatures.execute()
print('done')

print('Calculated first order features: ')
for (key, val) in six.iteritems(result):
  print('  ', key, ':', val)

Calculating first order features...
done
Calculated first order features: 
   Mean : 825.235436307


### Calculate Shape Features

In [10]:
shapeFeatures = shape.RadiomicsShape(croppedImage, croppedMask, **settings)

# Set the features to be calculated
# shapeFeatures.enableFeatureByName('Volume', True)
shapeFeatures.enableAllFeatures()

In [11]:
# Print out the docstrings of the enabled features
print('Will calculate the following shape features: ')
for f in shapeFeatures.enabledFeatures.keys():
  print(f)
  print(getattr(shapeFeatures, 'get%sFeatureValue' % f).__doc__)

Will calculate the following shape features: 
Maximum3DDiameter

    **8. Maximum 3D diameter**

    Maximum 3D diameter is defined as the largest pairwise Euclidean distance between surface voxels in the ROI.

    Also known as Feret Diameter.

    .. warning::
      This feature is only available when C Extensions are enabled
    
Maximum2DDiameterSlice

    **9. Maximum 2D diameter (Slice)**

    Maximum 2D diameter (Slice) is defined as the largest pairwise Euclidean distance between tumor surface voxels in
    the row-column (generally the axial) plane.

    .. warning::
      This feature is only available when C Extensions are enabled
    
Sphericity

    **4. Sphericity**

    .. math::
      \textit{sphericity} = \frac{\sqrt[3]{36 \pi V^2}}{A}

    Sphericity is a measure of the roundness of the shape of the tumor region relative to a sphere. It is a
    dimensionless measure, independent of scale and orientation. The value range is :math:`0 < sphericity \leq 1`, where
    a v

In [12]:
# Calculate the features and print(out result)
print('Calculating shape features...',)
result = shapeFeatures.execute()
print('done')

print('Calculated shape features: ')
for (key, val) in six.iteritems(result):
  print('  ', key, ':', val)

Calculating shape features...
done
Calculated shape features: 
   Maximum3DDiameter : 65.5366145873
   Maximum2DDiameterSlice : 47.2187913633
   Sphericity : 0.485061744222
   MinorAxis : 34.8497016669
   Elongation : 0.562117162717
   SurfaceVolumeRatio : 0.392308261863
   Volume : 16412.6586914
   MajorAxis : 61.9972204698
   SurfaceArea : 6438.82160378
   Flatness : 0.461059753466
   LeastAxis : 28.5844231854
   Maximum2DDiameterColumn : 44.5487904052
   Maximum2DDiameterRow : 61.5801767135


### Calculate GLCM Features

In [13]:
glcmFeatures = glcm.RadiomicsGLCM(croppedImage, croppedMask, **settings)

# Set the features to be calculated
# glcmFeatures.enableFeatureByName('SumEntropy', True)
glcmFeatures.enableAllFeatures()

In [14]:
# Print out the docstrings of the enabled features
print('Will calculate the following GLCM features: ')
for f in glcmFeatures.enabledFeatures.keys():
  print(f)
  print(getattr(glcmFeatures, 'get%sFeatureValue' % f).__doc__)

Will calculate the following GLCM features: 
JointAverage

    **2. Joint Average**

    .. math::
      \textit{joint average} = \mu_x = \displaystyle\sum^{N_g}_{i=1}\displaystyle\sum^{N_g}_{j=1}{p(i,j)i}

    Returns the mean gray level intensity of the :math:`i` distribution.

    .. warning::
      As this formula represents the average of the distribution of :math:`i`, it is independent from the
      distribution of :math:`j`. Therefore, only use this formula if the GLCM is symmetrical, where
      :math:`p_x(i) = p_y(j) \text{, where } i = j`.
    
SumAverage

    **21. Sum Average**

    .. math::

      \textit{sum average} = \displaystyle\sum^{2N_g}_{k=2}{p_{x+y}(k)k}

    Sum Average measures the relationship between occurrences of pairs
    with lower intensity values and occurrences of pairs with higher intensity
    values.

    .. warning::
      When GLCM is symmetrical, :math:`\mu_x = \mu_y`, and therefore :math:`\text{Sum Average} = \mu_x + \mu_y =
      2 \mu_x = 2 *

In [15]:
# Calculate the features and print(out result)
print('Calculating GLCM features...',)
result = glcmFeatures.execute()
print('done')

print('Calculated GLCM features: ')
for (key, val) in six.iteritems(result):
  print('  ', key, ':', val)

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Calculating GLCM features...
done
Calculated GLCM features: 
   JointAverage : 16.5538077244
   SumAverage : 33.1076154489
   JointEntropy : 8.79969627025
   ClusterShade : 19.6050834273
   MaximumProbability : 0.00735239226629
   Idmn : 0.961402169623
   JointEnergy : 0.00289314924299
   Contrast : 47.4921251144
   DifferenceEntropy : 3.74406097807
   InverseVariance : 0.198818841971
   DifferenceVariance : 16.6556370503
   Idn : 0.87260521574
   Idm : 0.200222556405
   Correlation : 0.39175220067
   Autocorrelation : 289.543699402
   SumEntropy : 5.35424132149
   SumSquares : 39.0558795922
   ClusterProminence : 27995.9375919
   Imc2 : 0.694224902067
   Imc1 : -0.0943893880874
   DifferenceAverage : 5.28446878987
   Id : 0.28722572383
   ClusterTendency : 108.731393255


### Calculate GLRLM Features

In [16]:
glrlmFeatures = glrlm.RadiomicsGLRLM(croppedImage, croppedMask, **settings)

# Set the features to be calculated
# glrlmFeatures.enableFeatureByName('ShortRunEmphasis', True)
glrlmFeatures.enableAllFeatures()

In [17]:
# Print out the docstrings of the enabled features
print('Will calculate the following GLRLM features: ')
for f in glrlmFeatures.enabledFeatures.keys():
  print(f)
  print(getattr(glrlmFeatures, 'get%sFeatureValue' % f).__doc__)

Will calculate the following GLRLM features: 
ShortRunLowGrayLevelEmphasis

    **13. Short Run Low Gray Level Emphasis (SRLGLE)**

    .. math::
      \textit{SRLGLE} = \frac{\sum^{N_g}_{i=1}\sum^{N_r}_{j=1}{\frac{\textbf{P}(i,j|\theta)}{i^2j^2}}}{N_z(\theta)}

    SRLGLE measures the joint distribution of shorter run lengths with lower gray-level values.
    
GrayLevelVariance

    **8. Gray Level Variance (GLV)**

    .. math::
      \textit{GLV} = \displaystyle\sum^{N_g}_{i=1}\displaystyle\sum^{N_r}_{j=1}{p(i,j|\theta)(i - \mu)^2}

    Here, :math:`\mu = \displaystyle\sum^{N_g}_{i=1}\displaystyle\sum^{N_r}_{j=1}{p(i,j|\theta)i}`

    GLV measures the variance in gray level intensity for the runs.
    
LowGrayLevelRunEmphasis

    **11. Low Gray Level Run Emphasis (LGLRE)**

    .. math::
      \textit{LGLRE} = \frac{\sum^{N_g}_{i=1}\sum^{N_r}_{j=1}{\frac{\textbf{P}(i,j|\theta)}{i^2}}}{N_z(\theta)}

    LGLRE measures the distribution of low gray-level values, with a higher value in

In [18]:
# Calculate the features and print(out result)
print('Calculating GLRLM features...',)
result = glrlmFeatures.execute()
print('done')

print('Calculated GLRLM features: ')
for (key, val) in six.iteritems(result):
  print('  ', key, ':', val)

Calculating GLRLM features...
done
Calculated GLRLM features: 
   ShortRunLowGrayLevelEmphasis : 0.00822976624416
   GrayLevelVariance : 39.118151022
   LowGrayLevelRunEmphasis : 0.00860039789166
   GrayLevelNonUniformityNormalized : 0.0451412381498
   RunVariance : 0.0847945778959
   GrayLevelNonUniformity : 175.635192315
   LongRunEmphasis : 1.22684403826
   ShortRunHighGrayLevelEmphasis : 268.974179841
   RunLengthNonUniformity : 3500.04323157
   ShortRunEmphasis : 0.955939173141
   LongRunHighGrayLevelEmphasis : 341.286579098
   RunPercentage : 0.940406463249
   LongRunLowGrayLevelEmphasis : 0.0106011704787
   RunEntropy : 4.91503800316
   HighGrayLevelRunEmphasis : 281.066493909
   RunLengthNonUniformityNormalized : 0.895049465948


### Calculate GLSZM Features

In [19]:
glszmFeatures = glszm.RadiomicsGLSZM(croppedImage, croppedMask, **settings)

# Set the features to be calculated
# glszmFeatures.enableFeatureByName('LargeAreaEmphasis', True)
glszmFeatures.enableAllFeatures()

In [20]:
# Print out the docstrings of the enabled features
print('Will calculate the following GLSZM features: ')
for f in glszmFeatures.enabledFeatures.keys():
  print(f)
  print(getattr(glszmFeatures, 'get%sFeatureValue' % f).__doc__)

Will calculate the following GLSZM features: 
GrayLevelVariance

    **8. Gray Level Variance (GLV)**

    .. math::
      \textit{GLV} = \displaystyle\sum^{N_g}_{i=1}\displaystyle\sum^{N_s}_{j=1}{p(i,j)(i - \mu)^2}

    Here, :math:`\mu = \displaystyle\sum^{N_g}_{i=1}\displaystyle\sum^{N_s}_{j=1}{p(i,j)i}`

    GLV measures the variance in gray level intensities for the zones.
    
SmallAreaHighGrayLevelEmphasis

    **14. Small Area High Gray Level Emphasis (SAHGLE)**

    .. math::
      \textit{SAHGLE} = \frac{\sum^{N_g}_{i=1}\sum^{N_s}_{j=1}{\frac{\textbf{P}(i,j)i^2}{j^2}}}{N_z}

    SAHGLE measures the proportion in the image of the joint distribution of smaller size zones with higher gray-level
    values.
    
GrayLevelNonUniformityNormalized

    **4. Gray Level Non-Uniformity Normalized (GLNN)**

    .. math::
      \textit{GLNN} = \frac{\sum^{N_g}_{i=1}\left(\sum^{N_s}_{j=1}{\textbf{P}(i,j)}\right)^2}{N_z^2}

    GLNN measures the variability of gray-level intensity values i

In [21]:
# Calculate the features and print(out result)
print('Calculating GLSZM features...',)
result = glszmFeatures.execute()
print('done')

print('Calculated GLSZM features: ')
for (key, val) in six.iteritems(result):
  print('  ', key, ':', val)

Calculating GLSZM features...
done
Calculated GLSZM features: 
   GrayLevelVariance : 40.6031399239
   ZoneVariance : 8.72123909749
   GrayLevelNonUniformityNormalized : 0.0440573079013
   SizeZoneNonUniformityNormalized : 0.399784380451
   SizeZoneNonUniformity : 747.596791444
   GrayLevelNonUniformity : 82.3871657754
   LargeAreaEmphasis : 13.6155080214
   SmallAreaHighGrayLevelEmphasis : 193.438051926
   ZonePercentage : 0.4520183708
   LargeAreaLowGrayLevelEmphasis : 0.127238415533
   LargeAreaHighGrayLevelEmphasis : 3514.76149733
   HighGrayLevelZoneEmphasis : 288.623529412
   SmallAreaEmphasis : 0.656447899959
   LowGrayLevelZoneEmphasis : 0.00910094202771
   ZoneEntropy : 6.5082149862
   SmallAreaLowGrayLevelEmphasis : 0.0064169820551


## Calculate Features using Laplacian of Gaussian Filter

Calculating features on filtered images is very similar to calculating features on the original image. All filters in PyRadiomics have the same input and output signature, and there is even one for applying no filter. This enables to loop over a list of requested filters and apply them in the same piece of code. It is applied like this in the execute function in feature extractor. The input for the filters is the image, with additional keywords. If no additional keywords are supplied, the filter uses default values where applicable. It returns a [generator object](https://docs.python.org/2/reference/simple_stmts.html?#yield), allowing to define the generators to be applied before the filters functions are actually called.

### Calculate Firstorder on LoG filtered images

In [22]:
logFeatures = {}
sigmaValues = [1.0, 3.0, 5.0]
for logImage, imageTypename, inputSettings in imageoperations.getLoGImage(image, mask, sigma=sigmaValues):
  logImage, croppedMask = imageoperations.cropToTumorMask(logImage, mask, bb)
  logFirstorderFeatures = firstorder.RadiomicsFirstOrder(logImage, croppedMask, **inputSettings)
  logFirstorderFeatures.enableAllFeatures()
  logFeatures[imageTypename] = logFirstorderFeatures.execute()

In [23]:
# Show result
for sigma, features in six.iteritems(logFeatures):
  for (key, val) in six.iteritems(features):
    laplacianFeatureName = '%s_%s' % (str(sigma), key)
    print('  ', laplacianFeatureName, ':', val)

   log-sigma-3-0-mm-3D_InterquartileRange : 103.158138275
   log-sigma-3-0-mm-3D_Skewness : -0.498386343995
   log-sigma-3-0-mm-3D_Uniformity : 0.0906478492348
   log-sigma-3-0-mm-3D_Median : -73.3129653931
   log-sigma-3-0-mm-3D_Energy : 56119386.4932
   log-sigma-3-0-mm-3D_RobustMeanAbsoluteDeviation : 43.3779243984
   log-sigma-3-0-mm-3D_MeanAbsoluteDeviation : 64.3312024633
   log-sigma-3-0-mm-3D_TotalEnergy : 222641609.012
   log-sigma-3-0-mm-3D_Maximum : 114.296691895
   log-sigma-3-0-mm-3D_RootMeanSquared : 116.469898461
   log-sigma-3-0-mm-3D_90Percentile : 13.9173410416
   log-sigma-3-0-mm-3D_Minimum : -354.335235596
   log-sigma-3-0-mm-3D_Entropy : 3.72121444058
   log-sigma-3-0-mm-3D_Range : 468.63192749
   log-sigma-3-0-mm-3D_Variance : 6720.06651871
   log-sigma-3-0-mm-3D_10Percentile : -197.017340088
   log-sigma-3-0-mm-3D_Kurtosis : 3.18336583197
   log-sigma-3-0-mm-3D_Mean : -82.7355469484
   log-sigma-1-0-mm-3D_InterquartileRange : 81.8767185211
   log-sigma-1-0-mm-3D_

## Calculate Features using Wavelet filter

### Calculate Firstorder on filtered images

In [24]:
waveletFeatures = {}
for decompositionImage, decompositionName, inputSettings in imageoperations.getWaveletImage(image, mask):
  decompositionImage, croppedMask = imageoperations.cropToTumorMask(decompositionImage, mask, bb)
  waveletFirstOrderFeaturs = firstorder.RadiomicsFirstOrder(decompositionImage, croppedMask, **inputSettings)
  waveletFirstOrderFeaturs.enableAllFeatures()

  print('Calculate firstorder features with ', decompositionName)
  waveletFeatures[decompositionName] = waveletFirstOrderFeaturs.execute()

Calculate firstorder features with  wavelet-HHH
Calculate firstorder features with  wavelet-HLL
Calculate firstorder features with  wavelet-LHL
Calculate firstorder features with  wavelet-LHH
Calculate firstorder features with  wavelet-LLH
Calculate firstorder features with  wavelet-LLL
Calculate firstorder features with  wavelet-HHL
Calculate firstorder features with  wavelet-HLH
Calculate firstorder features with  wavelet-LLL


In [25]:
# Show result
for decompositionName, features in six.iteritems(waveletFeatures):
  for (key, val) in six.iteritems(features):
    waveletFeatureName = '%s_%s' % (str(decompositionName), key)
    print('  ', waveletFeatureName, ':', val)

   wavelet-HLH_InterquartileRange : 44.1166822627
   wavelet-HLH_Skewness : -0.109634215846
   wavelet-HLH_Uniformity : 0.192969183516
   wavelet-HLH_Median : -0.788893809524
   wavelet-HLH_Energy : 7500010.61679
   wavelet-HLH_RobustMeanAbsoluteDeviation : 18.8328798407
   wavelet-HLH_MeanAbsoluteDeviation : 30.6419982661
   wavelet-HLH_TotalEnergy : 29754680.7917
   wavelet-HLH_Maximum : 195.252851267
   wavelet-HLH_RootMeanSquared : 42.5782863744
   wavelet-HLH_90Percentile : 48.8399961041
   wavelet-HLH_Minimum : -254.072696334
   wavelet-HLH_Entropy : 2.75501833527
   wavelet-HLH_Range : 449.325547601
   wavelet-HLH_Variance : 1812.89759077
   wavelet-HLH_10Percentile : -48.7973652241
   wavelet-HLH_Kurtosis : 5.96264006105
   wavelet-HLH_Mean : -0.113489262994
   wavelet-HHH_InterquartileRange : 20.3192746422
   wavelet-HHH_Skewness : -0.0688112737237
   wavelet-HHH_Uniformity : 0.382919979814
   wavelet-HHH_Median : 0.109117292789
   wavelet-HHH_Energy : 1241014.78701
   wavelet